## Descrição

## Importando bibliotecas

In [1]:
import pandas as pd
import unidecode

## Funções usadas para facilitar

In [2]:
def limpa_string(string: str) -> str:
    
    string_limpa = unidecode.unidecode(string).replace(' ','').lower()
    return string_limpa

## Acessando e formatando fontes externas

### Informação de continente
Arquivo gerado com o auxilio do ChatGPT:
- https://chat.openai.com/share/03e30b53-c1cb-41d9-8521-98c130cba817

In [3]:
arquivo = '../dados/external/pais_e_continente_2.txt'
df_continente = pd.read_csv(arquivo, delimiter='|')
df_continente.columns = ['pais', 'continente', 'country_code']
df_continente['pais_code'] = df_continente['pais'].apply(limpa_string)
df_continente.head()

,pais,continente,country_code,pais_code
0,Afeganistão,Ásia,AFG,afeganistao
1,África do Sul,África,ZAF,africadosul
2,Albânia,Europa,ALB,albania
3,Alemanha,Europa,DEU,alemanha
4,Andorra,Europa,AND,andorra


In [4]:
df_continente[df_continente['pais'].str.startswith('Fin')]

,pais,continente,country_code,pais_code
72,Finlândia,Europa,FIN,finlandia


In [5]:
df_continente[df_continente['country_code'] == 'AFG']

,pais,continente,country_code,pais_code
0,Afeganistão,Ásia,AFG,afeganistao


***

### Informação de inflação
- https://www.worldbank.org/en/research/brief/inflation-database
- type: Headline Consumer Price Inflation
- "Note: For those countries where annual data are not yet availalbe for 2022, annual averages of monthly or quarterly inflation in 2022 with available data or the projections by the IMF World Economic Outlook (October 2022) are included instead."

In [67]:
df_inflacao = pd.read_excel('../dados/external/Inflation-data.xlsx',
                           sheet_name='hcpi_a').fillna(0)

columns_filter = list(df_inflacao.columns[5:-2])
columns_filter.insert(0, 'Country Code')
df_inflacao = df_inflacao[columns_filter]
df_inflacao.head()

,Country Code,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,ABW,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-2.370000,0.420000,0.480000,-0.890000,-0.474,3.58,4.257,1.222000,0.744000,5.520000
1,AFG,25.510000,25.510000,-12.520000,-10.680000,10.230000,11.850000,0.600000,7.650000,5.000000,...,7.390000,4.670000,-0.660000,4.380000,4.976,0.63,2.302,5.443000,5.062000,0.000000
2,AGO,7.970000,5.780000,15.800000,15.670000,27.420000,29.000000,80.699997,69.010002,48.459999,...,8.782000,7.298000,9.159000,32.377731,29.844,19.63,17.079,21.024000,23.846111,23.826819
3,ALB,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.934749,1.621947,1.910179,1.291234,1.987,2.03,1.411,1.620887,2.041472,6.725203
4,ARE,21.984699,21.984699,21.984699,21.984699,21.984699,21.984699,21.984699,21.969700,13.043500,...,1.094026,2.344322,4.067132,1.619795,1.967,3.06,-1.931,-2.079403,0.180000,4.827889


In [68]:
df_inflacao_melted = df_inflacao.melt(id_vars=['Country Code'], value_vars=df_inflacao.columns[1:])
df_inflacao_melted.columns = ['country_code', 'ano', 'inflacao']
df_inflacao_melted = df_inflacao_melted.round(1)
df_inflacao_melted.head()

,country_code,ano,inflacao
0,ABW,1970,0.0
1,AFG,1970,25.5
2,AGO,1970,8.0
3,ALB,1970,0.0
4,ARE,1970,22.0


***

## Juntando informações

Lendo dados formatados de exportações

In [7]:
df_exportacoes = pd.read_csv('../dados/formatted/df_exportacao_2.csv', delimiter='|')
df_exportacoes['pais_code'] = df_exportacoes['pais_destino'].apply(limpa_string)
df_exportacoes.head()

,ano,pais_origem,pais_destino,tipo,quantidade,valor,pais_code
0,1970,Brasil,Afeganistão,Vinho de mesa,0,0,afeganistao
1,1970,Brasil,África do Sul,Vinho de mesa,0,0,africadosul
2,1970,Brasil,"Alemanha, República Democrática",Vinho de mesa,0,0,"alemanha,republicademocratica"
3,1970,Brasil,Angola,Vinho de mesa,0,0,angola
4,1970,Brasil,Anguilla,Vinho de mesa,0,0,anguilla


In [8]:
df_exportacoes[df_exportacoes['pais_destino'].str.startswith('Ru')].pais_destino.value_counts()

Russia                   52
Russia,  Federação da    52
Name: pais_destino, dtype: int64

### Juntando com informação de continente e country code

In [9]:
df_com_continente = pd.merge(df_exportacoes, df_continente,
        left_on='pais_code', right_on='pais_code',
        how='left')

In [10]:
df_com_continente[df_com_continente['continente'].isna()].pais_destino.unique()

array(['Outros(1)', 'Provisão de Navios e Aeronaves'], dtype=object)

##### Valores nulos

In [11]:
df_com_continente[(df_com_continente['continente'].isna()) 
                  & (df_com_continente['ano'] > 2006)].pais_destino.value_counts()

Outros(1)                         15
Provisão de Navios e Aeronaves    15
Name: pais_destino, dtype: int64

### Juntando informações com inflação

In [70]:
df_com_inflacao = pd.merge(df_com_continente, df_inflacao_melted,
        left_on=['country_code', 'ano'], right_on=['country_code', 'ano'],
        how='left')

##### Valores nulos

In [84]:
df_com_inflacao[df_com_inflacao['inflacao'].isna()].pais_destino.value_counts().sort_values()

Outros(1)                              52
Alemanha, República Democrática da     52
Provisão de Navios e Aeronaves         52
Taiwan                                 52
Jérsei                                 52
Guadalupe                              52
Faroé, Ilhas                           52
Falkland (Ilhas Malvinas)              52
Coreia do Norte                        52
Anguilla                               52
Falkland (Malvinas)                    52
Iugoslâvia                             52
Taiwan (FORMOSA)                      104
Bermudas                              104
Mônaco                                104
Alemanha, República Democrática       104
Ilhas Virgens                         104
Ilha de Man                           156
Guiana Francesa                       156
Cuba                                  156
Antilhas Holandesas                   156
Gibraltar                             208
Taiwan (Formosa)                      208
Name: pais_destino, dtype: int64

## Escrevendo dado "enriquecido"

In [83]:
df_com_inflacao[['ano', 'pais_origem', 'pais_destino', 'tipo', 'quantidade',
                'valor', 'continente', 'country_code', 'inflacao']].head()

,ano,pais_origem,pais_destino,tipo,quantidade,valor,continente,country_code,inflacao
0,1970,Brasil,Afeganistão,Vinho de mesa,0,0,Ásia,AFG,25.5
1,1970,Brasil,África do Sul,Vinho de mesa,0,0,África,ZAF,4.1
2,1970,Brasil,"Alemanha, República Democrática",Vinho de mesa,0,0,Europa,DDR,NaN
3,1970,Brasil,Angola,Vinho de mesa,0,0,África,AGO,8.0
4,1970,Brasil,Anguilla,Vinho de mesa,0,0,Américas,AIA,NaN


In [82]:
df_com_inflacao[['ano', 'pais_origem', 'pais_destino', 'tipo', 'quantidade',
                'valor', 'continente', 'country_code', 'inflacao']].to_csv('../dados/formatted/df_exportacao_3.csv', sep='|', encoding='utf-8', index=False)